In [ ]:
"""
Sample code for running experiments using mlflow.
"""

import cloudpickle,os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
from mlflow.models import infer_signature
import mlflow.sklearn


import logging
from refractio import get_local_dataframe

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    """
    :param
    actual
    pred
    :returns
    rmse, mae, r2
    """
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def score_and_dump_func(file_path):
    """
    :param
    file_path
    """
    def score_func(model, request):
        """
        :param
        model
        request
        :returns
        score_output
        """
        # Enter your custom score function here
        
        score_output="Success"
        return score_output
    
    with open(file_path, "wb") as out:
        cloudpickle.dump(score_func, out)
        
    
def experiment(alpha=0.5, l1_ratio=0.5):
    """
    Params:
    alpha: 0.5 (default)
    l1_ratio: 0.5 (default)
    """
    # Tracking URI set
    mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL", "http://mlflow-server"))
    
    # Setting experiment name
    mlflow.set_experiment(os.getenv("EXPERIMENT_NAME", "sample_experiment"))

    # Adding description to the experiment
    tags = {'mlflow.note.content': os.getenv("EXPERIMENT_DESCRIPTION", "sample_description")}

    # Read the input csv file from /data mount attached to this notebook pod
    input_csv_file = "/data/" + os.getenv("EXPERIMENT_DATASET")
    try:
        data = get_local_dataframe(input_csv_file)
    except Exception as e:
        logger.exception(f"Unable to read training/test CSV.\nError: {e}")
        
    if not os.getenv("EXPERIMENT_TARGET_COLUMN"):
        logger.exception("Please provide target column!")
        raise Exception("Please provide target column!")

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # Registering datasets with mlflow experiment run
    dataset = mlflow.data.from_pandas(data, source=input_csv_file)
    train_dataset = mlflow.data.from_pandas(train)
    test_dataset = mlflow.data.from_pandas(test)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop([os.getenv("EXPERIMENT_TARGET_COLUMN")], axis=1)
    test_x = test.drop([os.getenv("EXPERIMENT_TARGET_COLUMN")], axis=1)
    train_y = train[[os.getenv("EXPERIMENT_TARGET_COLUMN")]]
    test_y = test[[os.getenv("EXPERIMENT_TARGET_COLUMN")]]

    with mlflow.start_run(tags=tags):
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # logging params and metrics to the experiment runs
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        # Log input data to MLflow run artifact.
        mlflow.log_artifact(input_csv_file)

        # Registering datasets with mlflow experiment run
        mlflow.log_input(dataset, context="input")
        mlflow.log_input(train_dataset, context="train")
        mlflow.log_input(test_dataset, context="test")

        # Set custom tags
        mlflow.set_tags({
            "template_id": os.getenv("template_id", "sample_template_id"),
            "notebook_name": os.getenv("notebook_name", "sample_notebook_name"),
            "algorithm": "ElasticNet"       # Update this tag when trying with some other algos, this helps in filtering the runs based on algos.
        })

        predictions = lr.predict(train_x)
        signature = infer_signature(train_x, predictions)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        # Storing score function for the model
        score_and_dump_func("/tmp/scoring_func")       
        mlflow.log_artifact("/tmp/scoring_func")
        
        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(
                lr, "model", registered_model_name=os.getenv("EXPERIMENT_NAME", "sample_experiment"), signature=signature
            )
        else:
            mlflow.sklearn.log_model(lr, "model", signature=signature)
        
# Running Experiment with different alpha and l1_ratio params.
experiment(alpha=0.5, l1_ratio=0.5)